# Splits
Create a folds/train/valid csv file.   
Depend on the problematic file for labels.   
Depend on same file for case IDs, which are incomplete.     

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import csv

In [2]:
GIVEN='/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/cvfolds.csv'
LABFILE='/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/aug_case_labels.csv'
OUTFILE='/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/aug_train_valid_splits.csv'

In [3]:
df = pd.read_csv(GIVEN)
df

,f0_TRAIN_X,f0_TRAIN_y,f0_TEST_X,f0_TEST_y,f1_TRAIN_X,f1_TRAIN_y,f1_TEST_X,f1_TEST_y,f2_TRAIN_X,f2_TRAIN_y,f2_TEST_X,f2_TEST_y,f3_TRAIN_X,f3_TRAIN_y,f3_TEST_X,f3_TEST_y,f4_TRAIN_X,f4_TRAIN_y,f4_TEST_X,f4_TEST_y
0,TCGA-02-0004,0,TCGA-02-0430,0.0,TCGA-02-0010,0,TCGA-02-0004,0.0,TCGA-02-0004,0,TCGA-02-0010,0.0,TCGA-02-0004,0,TCGA-02-0439,0.0,TCGA-02-0004,0,TCGA-06-0185,0.0
1,TCGA-02-0010,0,TCGA-06-5412,0.0,TCGA-14-0789,0,TCGA-06-0129,0.0,TCGA-02-0430,0,TCGA-14-0789,0.0,TCGA-02-0010,0,TCGA-02-0446,0.0,TCGA-02-0010,0,TCGA-06-0201,0.0
2,TCGA-14-0789,0,TCGA-08-0517,0.0,TCGA-02-0025,0,TCGA-06-0189,0.0,TCGA-02-0439,0,TCGA-02-0025,0.0,TCGA-14-0789,0,TCGA-06-0190,0.0,TCGA-14-0789,0,TCGA-06-0211,0.0
3,TCGA-02-0025,0,TCGA-14-0787,0.0,TCGA-02-0033,0,TCGA-06-0241,0.0,TCGA-02-0446,0,TCGA-02-0033,0.0,TCGA-02-0025,0,TCGA-06-0214,0.0,TCGA-02-0025,0,TCGA-06-0879,0.0
4,TCGA-02-0033,0,TCGA-28-1746,0.0,TCGA-02-0285,0,TCGA-06-1084,0.0,TCGA-06-0129,0,TCGA-02-0285,0.0,TCGA-02-0033,0,TCGA-06-0878,0.0,TCGA-02-0033,0,TCGA-06-1801,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,TCGA-QH-A6X8,5,NaN,NaN,TCGA-DB-A64P,5,NaN,NaN,TCGA-QH-A65R,5,NaN,NaN,TCGA-DH-A66G,5,NaN,NaN,TCGA-QH-A65R,5,NaN,NaN
97,TCGA-S9-A6TW,5,NaN,NaN,TCGA-DH-A669,5,NaN,NaN,TCGA-QH-A6X8,5,NaN,NaN,TCGA-HW-A5KJ,5,NaN,NaN,TCGA-QH-A6X8,5,NaN,NaN
98,TCGA-S9-A6TX,5,NaN,NaN,TCGA-DH-A66G,5,NaN,NaN,TCGA-S9-A6TW,5,NaN,NaN,TCGA-QH-A65R,5,NaN,NaN,TCGA-S9-A6TW,5,NaN,NaN
99,TCGA-S9-A7J2,5,NaN,NaN,TCGA-QH-A6X8,5,NaN,NaN,TCGA-S9-A6TX,5,NaN,NaN,TCGA-S9-A6TW,5,NaN,NaN,TCGA-S9-A6TX,5,NaN,NaN


In [4]:
LABFILE='/home/jrm/Adjeroh/Glioma/August_Run/TrainValidSplits/aug_case_labels.csv'
with open (OUTFILE,'w') as outfile:
    mywriter = csv.writer(outfile)
    for idx,row in df.iterrows():
        mywriter.writerow( (row['f0_TRAIN_X'],str(int(row['f0_TRAIN_y']))))
    for idx,row in df.iterrows():
        if not pd.isnull(row['f0_TEST_X']):
            mywriter.writerow( (row['f0_TEST_X'],str(int(row['f0_TEST_y']))))

In [5]:
X=[]
y=[]
with open (LABFILE,'r') as labels:
    myreader = csv.reader(labels)
    for row in myreader:
        X.append(row[0])
        y.append(row[1])

In [6]:
# Reproducible stratified K-fold cross validation sets.
skf = StratifiedKFold(n_splits=5, random_state=12345, shuffle=True)
fold = 0
with open (OUTFILE,'w') as folds:
    mywriter = csv.writer(folds)
    row = ('Fold','Split','Case','Class')
    mywriter.writerow(row)
    for train_index, test_index in skf.split(X, y):
        for i in train_index:
            row = (fold,'Train',X[i],str(y[i]))
            mywriter.writerow(row)
        for i in test_index:
            row = (fold,'Valid',X[i],str(y[i]))
            mywriter.writerow(row)
        fold += 1